In [ ]:
# | default_exp mkdocs

In [ ]:
# | export

from typing import *

import logging
import os
import re
import collections
from pathlib import Path
import textwrap
import shutil
import types
import pkgutil
import importlib
import subprocess # nosec: B404

import typer
from typer.testing import CliRunner

from configupdater import ConfigUpdater, Section
from configupdater.option import Option

from configparser import ConfigParser

from fastcore.script import call_parse
from nbdev.serve import proc_nbs
from nbdev.process import NBProcessor
from nbdev.frontmatter import FrontmatterProc

import nbconvert

from nbdev_mkdocs._package_data import get_root_data_path
from nbdev_mkdocs._helpers.cli_doc import generate_cli_doc

In [ ]:
import pytest
import numpy as np
from tempfile import TemporaryDirectory
import yaml

# Helpers

In [ ]:
# | export

def _get_value_from_config(root_path: str, config_name: str) -> str:
    """Get the value from settings.ini file"""
    
    settings_path = Path(root_path) / "settings.ini"
    config = ConfigParser()
    config.read(settings_path)
    if not config.has_option("DEFAULT", config_name):
        return ""
    return config["DEFAULT"][config_name]

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)
    ret_val = _get_value_from_config(d, "lib_path")
    print(ret_val)
    assert ret_val == "nbdev_mkdocs"

nbdev_mkdocs


## Create new

### Add requirements to settings

In [ ]:
# | export


def _add_requirements_to_settings(root_path: str):
    """Adds requirments needed for mkdocs to settings.ini

    Params:
        root_path: path to where the settings.ini file is located

    """
    _requirements_path = get_root_data_path() / "requirements.txt"
    with open(_requirements_path, "r") as f:
        _new_req_to_add = f.read()
        lines = _new_req_to_add.split("\n")
        lines = [s.strip() for s in lines]
        lines = [s for s in lines if s != ""]
        _new_req_to_add = " \\\n".join(lines)

    setting_path = Path(root_path) / "settings.ini"
    if not setting_path.exists():
        typer.secho(
            f"Path '{setting_path.resolve()}' does not exists! Please use --root_path option to set path to setting.ini file.",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    try:

        updater = ConfigUpdater()
        updater.read(setting_path)
    except Exception as e:
        typer.secho(
            f"Error while reading '{setting_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=2)

    try:
        if "requirements" not in updater["DEFAULT"]:
            updater["DEFAULT"].last_block.add_after.space(2).comment("### Optional ###").option("requirements", "")  # type: ignore

        old_req: str = updater["DEFAULT"]["requirements"].value  # type: ignore

        def remove_leading_spaces(s: str) -> str:
            return "\n".join([x.lstrip() for x in s.split("\n")])

        old_req = remove_leading_spaces(old_req)
        new_req = remove_leading_spaces(_new_req_to_add)
        if new_req in old_req:
            typer.secho(f"Requirements already added to '{setting_path.resolve()}'.")
            return

        req = old_req + " \\\n" + new_req
        req = textwrap.indent(req, " " * 4)

        req_option = Option(key="requirements", value=req)
        updater["DEFAULT"]["requirements"] = req_option
    except Exception as e:
        typer.secho(
            f"Error while updating requiremets in '{setting_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

    updater.update_file()

    typer.secho(f"Requirements added to '{setting_path.resolve()}'.")

    return

In [ ]:
with TemporaryDirectory() as d:
    shutil.copyfile(Path("..") / "settings.ini", Path(d) / "settings.ini")

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["requirements"] = Option(
        key="requirements", value="\\\n  nbdev>=2.3.7 \\\n  typer[all]==0.6.1"
    )
    updater.update_file()

    assert "mkdocs" not in updater["DEFAULT"]["requirements"].value

    # testing adding requirements
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall("mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["requirements"].value)
    assert len(founded) == 5, founded

    # do nothin if the requirements are already added
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall("mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["requirements"].value)
    assert len(founded) == 5, founded

    print(updater)

Requirements added to '/tmp/tmpjxuo0ncp/settings.ini'.
Requirements already added to '/tmp/tmpjxuo0ncp/settings.ini'.
[DEFAULT]
# All sections below are required unless otherwise specified.
# See https://github.com/fastai/nbdev/blob/master/settings.ini for examples.

### Python library ###
repo = nbdev-mkdocs
lib_name = %(repo)s
version = 0.0.1rc2
min_python = 3.7
license = apache2

### nbdev ###
doc_path = _docs
lib_path = nbdev_mkdocs
nbs_path = nbs
recursive = True
tst_flags = notest
put_version_in_init = True
black_formatting = True

### Docs ###
branch = main
custom_sidebar = False
doc_host = https://%(user)s.github.io
doc_baseurl = /%(repo)s
git_url = https://github.com/%(user)s/%(repo)s
title = %(lib_name)s

### PyPI ###
audience = Developers
author = airt
author_email = info@airt.ai
copyright = 2022 onwards, %(author)s
description = Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto
keywords = nbdev jupyter notebook python mkdocs materia

### Create mkdocs dir

In [ ]:
# | export


def _create_mkdocs_dir(root_path: str):
    mkdocs_template_path = get_root_data_path() / "mkdocs_template"
    if not mkdocs_template_path.exists():
        typer.secho(
            f"Unexpected error: path {mkdocs_template_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=4)
    dst_path = Path(root_path) / "mkdocs"
    if dst_path.exists():
        typer.secho(
            f"Directory {dst_path.resolve()} already exist, skipping its creation.",
        )
    else:
        shutil.copytree(mkdocs_template_path, dst_path)
        #         shutil.move(dst_path.parent / "mkdocs_template", dst_path)
        typer.secho(
            f"Directory {dst_path.resolve()} created.",
        )

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_dir(d)

    print("\n".join([str(p) for p in (Path(d) / "mkdocs").glob("**/*")]))

Directory /tmp/tmpdgf1lwgb/mkdocs created.
/tmp/tmpdgf1lwgb/mkdocs/overrides
/tmp/tmpdgf1lwgb/mkdocs/docs
/tmp/tmpdgf1lwgb/mkdocs/site
/tmp/tmpdgf1lwgb/mkdocs/overrides/main.html
/tmp/tmpdgf1lwgb/mkdocs/docs/stylesheets
/tmp/tmpdgf1lwgb/mkdocs/docs/images
/tmp/tmpdgf1lwgb/mkdocs/docs/javascripts
/tmp/tmpdgf1lwgb/mkdocs/docs/stylesheets/extra.css
/tmp/tmpdgf1lwgb/mkdocs/docs/images/favicon.ico
/tmp/tmpdgf1lwgb/mkdocs/docs/javascripts/extra.js
/tmp/tmpdgf1lwgb/mkdocs/docs/javascripts/mathjax.js
/tmp/tmpdgf1lwgb/mkdocs/site/assets
/tmp/tmpdgf1lwgb/mkdocs/site/404.html
/tmp/tmpdgf1lwgb/mkdocs/site/assets/stylesheets
/tmp/tmpdgf1lwgb/mkdocs/site/assets/images
/tmp/tmpdgf1lwgb/mkdocs/site/assets/javascripts
/tmp/tmpdgf1lwgb/mkdocs/site/assets/_mkdocstrings.css
/tmp/tmpdgf1lwgb/mkdocs/site/assets/stylesheets/main.3de6f41f.min.css
/tmp/tmpdgf1lwgb/mkdocs/site/assets/stylesheets/palette.cc9b2e1e.min.css.map
/tmp/tmpdgf1lwgb/mkdocs/site/assets/stylesheets/main.3de6f41f.min.css.map
/tmp/tmpdgf1lw

### Create Mkdocs.yml

In [ ]:
# | export

_mkdocs_template_path = get_root_data_path() / "mkdocs_template.yml"

In [ ]:
assert _mkdocs_template_path.exists()

In [ ]:
# | export

with open(_mkdocs_template_path, "r") as f:
    _mkdocs_template = f.read()

In [ ]:
print(_mkdocs_template)

# Site
site_name: {title}
site_url: {doc_host}{doc_baseurl}
site_author: {author}
site_description: {description}
  
# Repository
repo_name: {repo}
repo_url: {git_url}
edit_uri: ""

copyright: {copyright}

docs_dir: docs
site_dir: site

plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
            - https://docs.python.org/3/objects.inv
        options:
            heading_level: 2
            show_category_heading: true
            show_root_heading: true
            show_root_toc_entry: true
            show_signature_annotations: true
            show_if_no_docstring: true
            
markdown_extensions:
    - pymdownx.arithmatex:
        generic: true
    - pymdownx.inlinehilite
    - pymdownx.details
    - pymdownx.emoji
    - pymdownx.magiclink
    - pymdownx.superfences
    - pymdownx.tasklist
    - pymdownx.highlight:
        linenums: false
    - pymdownx.snippets:
        check_paths: true
    - pymdownx.t

In [ ]:
# | export
def _get_kwargs_from_settings(
    settings_path: Path, mkdocs_template: Optional[str] = None
) -> Dict[str, str]:
    config = ConfigParser()
    config.read(settings_path)
    if not mkdocs_template:
        mkdocs_template = _mkdocs_template
    keys = [s[1:-1] for s in re.findall("\{.*?\}", _mkdocs_template)]
    kwargs = {k: config["DEFAULT"][k] for k in keys}
    return kwargs

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    kwargs = _get_kwargs_from_settings(settings_path)

    actual = _mkdocs_template.format(**kwargs)

kwargs

{'title': 'nbdev-mkdocs',
 'doc_host': 'https://airtai.github.io',
 'doc_baseurl': '/nbdev-mkdocs',
 'author': 'airt',
 'description': 'Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto',
 'repo': 'nbdev-mkdocs',
 'git_url': 'https://github.com/airtai/nbdev-mkdocs',
 'copyright': '2022 onwards, airt'}

In [ ]:
# | export


def _create_mkdocs_yaml(root_path: str):
    try:
        # create mkdocs folder if necessary
        mkdocs_path = Path(root_path) / "mkdocs" / "mkdocs.yml"
        mkdocs_path.parent.mkdir(exist_ok=True)
        # mkdocs.yml already exists, just return
        if mkdocs_path.exists():
            typer.secho(
                f"Path '{mkdocs_path.resolve()}' exists, skipping generation of it."
            )
            return

        # get default values from settings.ini
        settings_path = Path(root_path) / "settings.ini"
        kwargs = _get_kwargs_from_settings(settings_path)
        mkdocs_yaml_str = _mkdocs_template.format(**kwargs)
        with open(mkdocs_path, "w") as f:
            f.write(mkdocs_yaml_str)
            typer.secho(f"File '{mkdocs_path.resolve()}' generated.")
            return
    except Exception as e:
        typer.secho(
            f"Unexpected Error while creating '{mkdocs_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)

    with open(Path(d) / "mkdocs/mkdocs.yml") as f:
        y = yaml.safe_load(f)

y

File '/tmp/tmpzb287bwd/mkdocs/mkdocs.yml' generated.


{'site_name': 'nbdev-mkdocs',
 'site_url': 'https://airtai.github.io/nbdev-mkdocs',
 'site_author': 'airt',
 'site_description': 'Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto',
 'repo_name': 'nbdev-mkdocs',
 'repo_url': 'https://github.com/airtai/nbdev-mkdocs',
 'edit_uri': '',
 'copyright': '2022 onwards, airt',
 'docs_dir': 'docs',
 'site_dir': 'site',
 'plugins': [{'literate-nav': {'nav_file': 'SUMMARY.md'}},
  'search',
  {'mkdocstrings': {'handlers': {'python': {'import': ['https://docs.python.org/3/objects.inv'],
      'options': {'heading_level': 2,
       'show_category_heading': True,
       'show_root_heading': True,
       'show_root_toc_entry': True,
       'show_signature_annotations': True,
       'show_if_no_docstring': True}}}}}],
 'markdown_extensions': [{'pymdownx.arithmatex': {'generic': True}},
  'pymdownx.inlinehilite',
  'pymdownx.details',
  'pymdownx.emoji',
  'pymdownx.magiclink',
  'pymdownx.superfences',
  'pymdo

### Create summary_template.txt

In [ ]:
# | export

_summary_template = """- [Home](index.md)
{guides}
{api}
{cli}
{changelog}
"""

def _create_summary_template(root_path: str):
    try:
        # create mkdocs folder if necessary
        summary_template_path = Path(root_path) / "mkdocs" / "summary_template.txt"
        summary_template_path.parent.mkdir(exist_ok=True)
        # summary_template_path.yml already exists, just return
        if summary_template_path.exists():
            typer.secho(
                f"Path '{summary_template_path.resolve()}' exists, skipping generation of it."
            )
            return

        # generated a new summary_template_path.yml file
        with open(summary_template_path, "w") as f:
            f.write(_summary_template)
            typer.secho(f"File '{summary_template_path.resolve()}' generated.")
            return
    except Exception as e:
        typer.secho(
            f"Unexpected Error while creating '{summary_template_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)
    _create_summary_template(d)

    guides = """- Guides
    - [Guide one](docs/guide_1.md)"""
    
    api = """- API
    - [numpy.array](api/numpy/array.md)"""
    
    cli = """- CLI
    - [my-cli](cli/my_cli.md)"""
    
    changelog = "- [Releases](CHANGELOG.md)"
    
    with open(Path(d) / "mkdocs/summary_template.txt") as f:
        summary_template = f.read()
        summary = summary_template.format(guides=guides, api=api, cli=cli, changelog=changelog)
#         y = yaml.safe_load(summary)

print(summary)

File '/tmp/tmp16e_syd7/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp16e_syd7/mkdocs/summary_template.txt' generated.
- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- API
    - [numpy.array](api/numpy/array.md)
- CLI
    - [my-cli](cli/my_cli.md)
- [Releases](CHANGELOG.md)



### Bringing it all together

In [ ]:
# | export


def new(root_path: str):
    """Initialize mkdocs project files

    Creates **mkdocs** directory in the **root_path** directory and populates
    it with initial values. You should edit mkdocs.yml file to customize it if
    needed.

    Params:
        root_path: path under which mkdocs directory will be created
    """
    _add_requirements_to_settings(root_path)
    _create_mkdocs_dir(root_path)
    _create_mkdocs_yaml(root_path)
    _create_summary_template(root_path)
    
@call_parse
def new_cli(root_path: str):
    """Initialize mkdocs project files

    Creates **mkdocs** directory in the **root_path** directory and populates
    it with initial values. You should edit mkdocs.yml file to customize it if
    needed.
    """
    new(root_path)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    mkdocs_path = Path(d) / "mkdocs"
    assert settings_path.exists()
    assert mkdocs_path.exists()
    assert (mkdocs_path / "mkdocs.yml").exists()
    assert (mkdocs_path / "overrides" / "main.html").exists()
    assert (mkdocs_path / "site").exists()
    assert (mkdocs_path / "summary_template.txt").exists()
    

Requirements already added to '/tmp/tmp5xzd05ut/settings.ini'.
Directory /tmp/tmp5xzd05ut/mkdocs created.
File '/tmp/tmp5xzd05ut/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp5xzd05ut/mkdocs/summary_template.txt' generated.


## Build

### Build markdown files

In [ ]:
(Path("/tmp") / "/" / "///////console.html").resolve()

Path('/console.html')

In [ ]:
!ls /tmp

_docs/        fish.root/            tmpg_5zcqs5.less
fish.harish/  tmp2imt9_a__kernels/  tmplobsjxr1.less


In [ ]:
# | export

def _get_nbs_for_markdown_conversion(cache: Path):
    """Get a list of notebooks that needs to be converted to markdown.
    
    Args:
        cache: Path to the nbs cache folder
    """
    return list(cache.glob("index.ipynb")) + list(cache.glob("./guides/*.ipynb"))

In [ ]:
with TemporaryDirectory() as d:
    cache = proc_nbs()
    nbs = _get_nbs_for_markdown_conversion(cache)
    print(nbs)
    assert len(nbs) > 0, f"{len(nbs)=}"

[Path('/tf/nbdev-mkdocs/_proc/index.ipynb'), Path('/tf/nbdev-mkdocs/_proc/guides/Guide_00_Installation.ipynb'), Path('/tf/nbdev-mkdocs/_proc/guides/Guide_01_End_To_End_Walkthrough.ipynb')]


In [ ]:
# | export


def _generate_markdown_from_nbs(root_path: str):
    doc_path = Path(root_path) / "mkdocs" / "docs"
    doc_path.mkdir(exist_ok=True, parents=True)

    cache = proc_nbs()
    notebooks = _get_nbs_for_markdown_conversion(cache)

    for nb in notebooks:
        dir_prefix = str(nb.parent)[len(str(cache)) + 1 :]
        md = doc_path / f"{dir_prefix}" / f"{nb.stem}.md"
        md.parent.mkdir(parents=True, exist_ok=True)

#         cmd = f"quarto render {nb} -o {cache / f'{nb.stem}.md'} -t gfm --no-execute"
        try:
            cmd = f"quarto render --help"
            # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
            sp = subprocess.run(  # nosec: B602:subprocess_popen_with_shell_equals_true
                cmd,
                shell=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                check=True,
            )
            
            cmd = f"quarto render {nb} -o {cache / f'{nb.stem}.md'} -t gfm --no-execute"
            # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
            sp = subprocess.run(  # nosec: B602:subprocess_popen_with_shell_equals_true
                cmd,
                shell=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                check=True,
            )
        except subprocess.CalledProcessError as exc:
            raise ValueError(f"CMD Failed Returned {exc.returncode=}\n{exc=}\n{cmd=}")
        except Exception as exc:
            raise ValueError(f"CMD Failed Returned {exc=}\n{cmd=}")
        #         print(sp.stdout)
        #         if sp.returncode != 0:
        #             logging.exception(f"Command '{cmd}' failed!, {cmd=}")
        #             typer.secho(
        #                 f"Command '{cmd}' failed!",
        #                 err=True,
        #                 fg=typer.colors.RED,
        #             )
        #             raise typer.Exit(5)

        _md_cache = cache / "_docs" / f"{nb.stem}.md"
        shutil.move(_md_cache, md)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    _generate_markdown_from_nbs(root_path=d)
    
    # check markdown files
    print("Checks:")
    mds = list((Path(d)/"mkdocs"/"docs").glob("**/*.md"))
    print("\n".join([str(md) for md in mds]))
    assert len(mds) > 0, len(mds)

Requirements already added to '/tmp/tmp8bsxuuci/settings.ini'.
Directory /tmp/tmp8bsxuuci/mkdocs created.
File '/tmp/tmp8bsxuuci/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp8bsxuuci/mkdocs/summary_template.txt' generated.
Checks:
/tmp/tmp8bsxuuci/mkdocs/docs/index.md
/tmp/tmp8bsxuuci/mkdocs/docs/guides/Guide_01_End_To_End_Walkthrough.md
/tmp/tmp8bsxuuci/mkdocs/docs/guides/Guide_00_Installation.md


In [ ]:
# | export


def _replace_all(text: str, dir_prefix: str) -> str:
    """Replace the images relative path in the markdown text

    Args:
        text: String to replace
        dir_prefix: Sub directory prefix to append to the image's relative path

    Returns:
        The text with the updated images relative path
    """
    _replace = {}
    _pattern = re.compile(r"!\[[^\]]*\]\((.*?)\s*(\"(?:.*[^\"])\")?\s*\)")
    _matches = [match.groups()[0] for match in _pattern.finditer(text)]

    if len(_matches) > 0:
        for m in _matches:
            _replace[m] = (
                os.path.normpath(Path("../images/nbs/").joinpath(f"{dir_prefix}/{m}"))
                if len(dir_prefix) > 0
                else f"images/nbs/{m}"
            )

        for k, v in _replace.items():
            text = text.replace(k, v)

    return text

In [ ]:
text = """![Git Repo_Clone_Page](../img/test.png)
![Git Repo_Clone_Page](images/git_repo_clone_page.png)
"""

expected = """![Git Repo_Clone_Page](../images/nbs/img/test.png)
![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)
"""

dir_prefix = "guides"
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected

text = """![Git Repo_Clone_Page](img/test.png)
![Git Repo_Clone_Page](guides/images/git_repo_clone_page.png)
"""

expected = """![Git Repo_Clone_Page](images/nbs/img/test.png)
![Git Repo_Clone_Page](images/nbs/guides/images/git_repo_clone_page.png)
"""

dir_prefix = ""
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected

In [ ]:
# | export

def _update_path_in_markdown(cache: Path, doc_path: Path):
    """Update guide images relative path in the markdown files
    
    Args:
        cache: Path to the nbs cache directory
        doc_path: Path to the mkdocs/docs directory
    """
    notebooks = _get_nbs_for_markdown_conversion(cache)
    
    for nb in notebooks:
        dir_prefix = str(nb.parent)[len(str(cache)) + 1 :]
        md = doc_path / f"{dir_prefix}" / f"{nb.stem}.md"
        
        with open(Path(md), "r") as f:
            _new_text = f.read()
            _new_text = _replace_all(_new_text, dir_prefix)
        with open(Path(md), "w") as f:
            f.write(_new_text)


def _copy_guide_images_to_docs_dir(root_path: str):
    """Copy guide images to the docs directory

    Args:
        root_path: path under which mkdocs directory will be created
    """
    # Reference: https://github.com/quarto-dev/quarto-cli/blob/main/src/core/image.ts#L38
    image_extensions = [
        ".apng",
        ".avif",
        ".gif",
        ".jpg",
        ".jpeg",
        ".jfif",
        ".pjpeg",
        ".pjp",
        ".png",
        ".svg",
        ".webp",
    ]

    cache = proc_nbs()
    nbs_images_path = [
        p for p in Path(cache).glob(r"**/*") if p.suffix in image_extensions
    ]

    if len(nbs_images_path) > 0:
        doc_path = Path(root_path) / "mkdocs" / "docs" 
        img_path = Path(doc_path) / "images" / "nbs"
        for src_path in nbs_images_path:
            dir_prefix = str(src_path.parent)[len(str(cache)) + 1 :]
            dst_path = Path(img_path) / f"{dir_prefix}"
            dst_path.mkdir(exist_ok=True, parents=True)
            shutil.copy(src_path, dst_path)

        _update_path_in_markdown(cache, doc_path)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    _generate_markdown_from_nbs(root_path=d)
    _copy_guide_images_to_docs_dir(d)
    
    # check image files
    print("Checks for images:")
    imgs = list((Path(d)/ "mkdocs" / "docs" / "images" / "nbs").glob("**/*.*"))
    print("\n".join([str(img) for img in imgs]))
    assert len(imgs) > 0, len(imgs)
    
    # check markdown files
    print("Checks for markdown:")
    mds = list((Path(d)/"mkdocs"/"docs").glob("**/*.md"))
    print("\n".join([str(md) for md in mds]))
    
    for md in mds:
       with open(md, "r") as f:
            print(f.read())

### Build summary for guides

In [ ]:
# | export


def _get_title_from_notebook(nb_name: str) -> str:
    cache = proc_nbs()
    nb_path = Path(cache) / "guides" / f"{nb_name}.ipynb"
    
    if not nb_path.exists():
        typer.secho(
            f"Unexpected error: path {nb_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)
    
    nbp = NBProcessor(nb_path, procs=FrontmatterProc)
    nbp.process()
    return nbp.nb.frontmatter_["title"]

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)
    
    new(d)
    _generate_markdown_from_nbs(d)
        
    mds = sorted([md for md in Path(d).glob("**/*.md") if md.name.lower().startswith("guide")])
    for m in mds:
        title = _get_title_from_notebook(m.stem)
        print(title)
        assert len(title)

In [ ]:
# | export


def _generate_summary_for_guides(root_path: str) -> str:
    doc_path = Path(root_path) / "mkdocs" / "docs"
    mds = sorted([md for md in doc_path.glob("**/*.md") if md.name.lower().startswith("guide")])

    i = len(doc_path.parts)
    if len(mds) > 0:
        return "- Guides\n    - " + "    - ".join(
            [f"[{_get_title_from_notebook(md.stem)}]({'/'.join(md.parts[i:])})\n" for md in mds]
        )
    else:
        return ""

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    _generate_markdown_from_nbs(root_path=d)
    guides = _generate_summary_for_guides(root_path=d)
    
print(guides)

### Build API

In [ ]:
# | export


def get_submodules(package_name: str) -> List[str]:
    # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
    m = importlib.import_module(package_name)
    submodules = [
        info.name
        for info in pkgutil.walk_packages(m.__path__, prefix=f"{package_name}.")
    ]
    submodules = [
        x
        for x in submodules
        if not any([name.startswith("_") for name in x.split(".")])
    ]
    return submodules

In [ ]:
submodules = get_submodules("mkdocs")
submodules

In [ ]:
# | export


def generate_api_doc_for_submodule(root_path: str, submodule: str) -> str:
    subpath = "API/" + submodule.replace(".", "/") + ".md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)
    with open(path, "w") as f:
        f.write(f"::: {submodule}")
    subnames = submodule.split(".")
    if len(subnames) > 2:
        return " " * 4 * (len(subnames) - 2) + f"- [{subnames[-1]}]({subpath})"
    else:
        return f"- [{submodule}]({subpath})"


def generate_api_docs_for_module(root_path: str, module_name: str) -> str:
    submodules = get_submodules(module_name)
    shutil.rmtree(Path(root_path) / "mkdocs" / "docs" / "API", ignore_errors=True)
    submodule_summary = "\n".join(
        [
            generate_api_doc_for_submodule(root_path=root_path, submodule=x)
            for x in submodules
        ]
    )
    return "- API\n" + textwrap.indent(submodule_summary, prefix=" " * 4)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    api_summary = generate_api_docs_for_module(d, "mkdocs")
    print(api_summary)

    # make sure all paths exist
    paths = re.findall("\(.*?\)", api_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path

In [ ]:
# | export


def _restrict_line_length(s: str, width: int = 80) -> str:
    """Restrict the line length of the given string.

    Args:
        s: Docstring to fix the width
        width: The maximum allowed line length

    Returns:
        A new string in which each line is less than the specified width.
    """
    _s = ""

    for blocks in s.split("\n\n"):
        sub_block = blocks.split("\n  ")
        for line in sub_block:
            line = line.replace("\n", " ")
            line = "\n".join(textwrap.wrap(line, width=width, replace_whitespace=False))
            if len(sub_block) == 1:
                _s += line + "\n\n"
            else:
                _s += "\n" + line + "\n" if line.endswith(":") else " " + line + "\n"
    return _s

In [ ]:
s = """usage: nbdev_mkdocs_new [-h] root_path

Initialize mkdocs project files Creates **mkdocs** directory in the **root_path** directory and populates it with
initial values. You should edit mkdocs.yml file to customize it if needed.

positional arguments:
  root_path

optional arguments:
  -h, --help  show this help message and exit show this help message and exit show this help message and exit
  -h, --help  show this help message and exit
  --port PORT
  --domain DOMAIN
"""

width = 60
doc = _restrict_line_length(s, width)

print(doc)
assert all([len(line) <= width for line in doc.splitlines() if line.strip() != ''])

In [ ]:
# | export


def generate_cli_doc_for_submodule(root_path: str, cmd: str) -> str:

    cli_app_name = cmd.split("=")[0]
    module_name = cmd.split("=")[1].split(":")[0]
    method_name = cmd.split("=")[1].split(":")[1]

    subpath = f"CLI/{cli_app_name}.md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)

    # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
    m = importlib.import_module(module_name)
    if isinstance(getattr(m, method_name), typer.Typer):
        app = typer.Typer()
        app.command()(generate_cli_doc)
        runner = CliRunner()
        result = runner.invoke(app, [module_name, cli_app_name])
        cli_doc = str(result.stdout)
    else:
        cmd = f"{cli_app_name} --help"
        print(f"Not a typer command. Documenting: {cmd=}")

        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        cli_doc = subprocess.run(  # nosec: B602:subprocess_popen_with_shell_equals_true
            cmd,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        ).stdout.decode("utf-8")
        
        cli_doc = _restrict_line_length(cli_doc)
        cli_doc = "\n```\n" + cli_doc + "\n```\n"

    with open(path, "w") as f:
        f.write(cli_doc)

    return f"- [{cli_app_name}]({subpath})"


def generate_cli_docs_for_module(root_path: str, module_name: str) -> str:
    shutil.rmtree(Path(root_path) / "mkdocs" / "docs" / "CLI", ignore_errors=True)
    console_scripts = _get_value_from_config(root_path, "console_scripts")

    if not console_scripts:
        return ""

    submodule_summary = "\n".join(
        [
            generate_cli_doc_for_submodule(root_path=root_path, cmd=cmd)
            for cmd in console_scripts.split("\n")
        ]
    )

    return "- CLI\n" + textwrap.indent(submodule_summary, prefix=" " * 4)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    cli_summary = generate_cli_docs_for_module(d, "nbdev_mkdocs")
    print(cli_summary)

    # make sure all paths and content exist
    paths = re.findall("\(.*?\)", cli_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path

In [ ]:
# | export

def _copy_change_log_if_exists(root_path: Union[Path, str], docs_path: Union[Path, str]) -> str:
    changelog = ""
    source_change_log_path = Path(root_path) / "CHANGELOG.md"
    dst_change_log_path = Path(docs_path) / "CHANGELOG.md"
    if source_change_log_path.exists():
        shutil.copy(source_change_log_path, dst_change_log_path)
        changelog = "- [Releases](CHANGELOG.md)"
    return changelog

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)
    
    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")
    
    print(f"{change_log=}")
    assert change_log == ""
    
    change_log_path = Path(d) / "CHANGELOG.md"
    with open(change_log_path, "w") as f:
        f.write("CHANGELOG")
        
    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")
    
    print(f"{change_log=}")
    assert change_log == "- [Releases](CHANGELOG.md)"


### Brining it all together

In [ ]:
# | export

def build_summary(
    root_path: str,
    module: str,
):
    # create docs_path if needed
    docs_path = Path(root_path) / "mkdocs" / "docs"
    docs_path.mkdir(exist_ok=True)

    # copy README.md as index.md
    shutil.copy(Path(root_path) / "README.md", docs_path / "index.md")

    # generate markdown files
    _generate_markdown_from_nbs(root_path)
    
    # copy guide images to docs dir and update path in generated markdown files
    _copy_guide_images_to_docs_dir(root_path)
    
    # generates guides
    guides = _generate_summary_for_guides(root_path)
    
    # generate API
    api = generate_api_docs_for_module(root_path, module)
    
    # generate CLI
    cli = generate_cli_docs_for_module(root_path, module)
    
    # copy CHANGELOG.md as CHANGELOG.md is exists
    changelog = _copy_change_log_if_exists(root_path, docs_path)

    # read summary template from file
    with open(Path(root_path) / "mkdocs" / "summary_template.txt") as f:
        summary_template = f.read()
        
    summary = summary_template.format(guides=guides, api=api, cli=cli, changelog=changelog)
    summary = "\n".join([l for l in [l.rstrip() for l in summary.split("\n")] if l != ""])
    
    with open(docs_path / "SUMMARY.md", mode="w") as f:
        f.write(summary)


In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    for fname in ["settings.ini", "README.md", "CHANGELOG.md"]:
        shutil.copyfile(Path("..") / fname, Path(d) / fname)

    new(d)

    build_summary(d, "nbdev_mkdocs")
    
    with open(Path(d) / "mkdocs/docs/SUMMARY.md") as f:
        summary = f.read()

    print(summary)

### Copy CNAME if needed

In [ ]:
# | export


def copy_cname_if_needed(root_path: str):
    cname_path = Path(root_path) / "CNAME"
    dst_path = Path(root_path) / "mkdocs" / "docs" / "CNAME"
    if cname_path.exists():
        dst_path.parent.mkdir(exist_ok=True, parents=True)
        shutil.copyfile(cname_path, dst_path)
        typer.secho(
            f"File '{cname_path.resolve()}' copied to '{dst_path.resolve()}'.",
        )
    else:
        typer.secho(
            f"File '{cname_path.resolve()}' not found, skipping copying..",
        )

In [ ]:
for has_cname in [True, False]:
    with TemporaryDirectory() as d:
        settings_path = Path(d) / "settings.ini"
        for fname in ["settings.ini", "README.md"] + ["CNAME"] if has_cname else []:
            shutil.copyfile(Path("..") / fname, Path(d) / fname)

        copy_cname_if_needed(d)
        if has_cname:
            assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists()
        else:
            assert not (Path(d) / "mkdocs" / "docs" / "CNAME").exists()

In [ ]:
# | export


def prepare(root_path: str):
    """Prepares mkdocs for serving

    Params:
        root_path: path under which mkdocs directory will be created
    """
    # copy cname if it exists
    copy_cname_if_needed(root_path)
    
    # get lib name from settings.ini
    lib_name = _get_value_from_config(root_path, "lib_name")
    lib_path = _get_value_from_config(root_path, "lib_path")

    build_summary(root_path, lib_path)

    cmd = f"mkdocs build -f {root_path}/mkdocs/mkdocs.yml"
    
    # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
    sp = subprocess.run( # nosec: B602:subprocess_popen_with_shell_equals_true
        cmd,
        shell=True,
        #         check=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )
    print(sp.stdout)
    if sp.returncode != 0:
        typer.secho(
            f"Command '{cmd}' failed!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(5)
        
        
@call_parse
def prepare_cli(root_path: str):
    """Prepares mkdocs for serving"""
    prepare(root_path)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    for fname in ["settings.ini", "README.md", "CNAME", "CHANGELOG.md"]:
        shutil.copyfile(Path("..") / fname, Path(d) / fname)

    new(d)

    prepare(d)
    
    !ll {d}/mkdocs/

    assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists
    assert (Path(d) / "mkdocs" / "docs" / "API").exists
    assert (Path(d) / "mkdocs" / "docs" / "SUMMARY.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "CHANGELOG.md").exists
#     !ls {d}/mkdocs/docs

## Preview

In [ ]:
# | export

import shlex


def preview(root_path: str, port: Optional[int] = None):
    """Previes mkdocs documentation

    Params:
        root_path: path under which mkdocs directory will be created
        port: port to use
    """
    cmd = f"mkdocs serve -f {root_path}/mkdocs/mkdocs.yml -a 0.0.0.0"
    if port:
        cmd = cmd + f":{port}"

    with subprocess.Popen( #nosec B603:subprocess_without_shell_equals_true
        shlex.split(cmd),
        stdout=subprocess.PIPE,
        bufsize=1,
        text=True,
        universal_newlines=True,
    ) as p:
        for line in p.stdout:  # type: ignore
            print(line, end="")

    if p.returncode != 0:
        typer.secho(
            f"Command '{cmd}' failed!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(6)
        
        
@call_parse
def preview_cli(root_path: str, port: Optional[int] = None):
    """Previes mkdocs documentation
    """
    preview(root_path, port)

In [ ]:
# | notest

with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    for fname in ["settings.ini", "README.md", "CNAME", "CHANGELOG.md"]:
        shutil.copyfile(Path("..") / fname, Path(d) / fname)

    new(d)

    prepare(d)


    assert (Path(d) / "mkdocs" / "docs" / "API").exists
    assert (Path(d) / "mkdocs" / "docs" / "SUMMARY.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists
    assert (Path(d) / "mkdocs" / "docs" / "CHANGELOG.md").exists
    !ls {d}/mkdocs/docs
    
    preview(d, port=4000)
    